In [29]:
path_xlsx_parte = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [30]:
import sys
sys.path.append('../_amigocloud')

In [31]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [32]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [33]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [34]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [35]:
df_lib = pd.DataFrame(data)

In [36]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,338,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
1,339,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
2,214,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
3,216,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
4,217,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,867,125,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 16:45:00+00:00,2024-08-21 17:00:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
72,870,123,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 18:20:00+00:00,2024-08-21 18:40:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
73,871,123,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 18:20:00+00:00,2024-08-21 18:40:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
74,868,124,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 17:15:00+00:00,2024-08-21 18:00:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None


In [37]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125]

In [38]:
lista_id = list(set(df_lib['id']))
lista_id

[659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 25,
 789,
 670,
 671,
 672,
 675,
 805,
 679,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 821,
 851,
 703,
 852,
 834,
 836,
 853,
 839,
 714,
 842,
 844,
 845,
 855,
 591,
 848,
 593,
 338,
 339,
 594,
 595,
 214,
 854,
 856,
 216,
 217,
 857,
 858,
 597,
 861,
 862,
 864,
 860,
 866,
 867,
 868,
 869,
 870,
 871]

In [39]:
nulos = df_lib.isnull().sum()
nulos

id                      0
idd                     0
fecha_registro          0
piloto_1                0
piloto_2                0
hora_inicio             0
hora_fin                0
labor                   0
temperatura             0
viento                  0
humedad                 0
estado_de_operacion     0
dron                    0
obs                    76
dtype: int64

In [40]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [41]:
if null_check:
    df_lib = None
else:
    None

In [42]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,338,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
1,339,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
2,214,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
3,216,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
4,217,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,867,125,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 16:45:00+00:00,2024-08-21 17:00:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
72,870,123,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 18:20:00+00:00,2024-08-21 18:40:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
73,871,123,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 18:20:00+00:00,2024-08-21 18:40:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None
74,868,124,2024-08-21 04:00:00+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 17:15:00+00:00,2024-08-21 18:00:00+00:00,LIBERACION,31,22,63,CONCLUIDO,MT 02,None


In [43]:
df_parte = pd.read_excel(path_xlsx_parte)

In [44]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE
0,641,2024-05-23,21.0,8,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERAZANZA - SUAREZ DAVID,L1,17.000000,...,DA 02,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
1,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L9.2,17.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
2,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L13,10.710000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
3,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C10.3,2.490000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
4,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C12.2,0.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,NaN,NaT,NaN,0,2765,DAVALOS GUARISTI MARIO,1714,PENOCO--DAVALOS MARIO,L6,2.705832,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,867.0,60.0,NaN,NaN
855,NaN,NaT,NaN,0,2765,DAVALOS GUARISTI MARIO,990,MADREJON--MARIO DAVALOS,L9.3,0.804385,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,868.0,60.0,NaN,NaN
856,NaN,NaT,NaN,0,2765,DAVALOS GUARISTI MARIO,990,MADREJON--MARIO DAVALOS,L5,2.060354,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,869.0,60.0,NaN,NaN
857,NaN,NaT,NaN,0,2765,DAVALOS GUARISTI MARIO,1204,TOCAYO--MARIO DAVALOS,L2.1,1.130436,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,870.0,60.0,NaN,NaN


In [45]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [46]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22988\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [47]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [48]:
lista_idd

[109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125]

In [49]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 109
se proceso IDD: 110
se proceso IDD: 111
se proceso IDD: 112
se proceso IDD: 113
se proceso IDD: 114
se proceso IDD: 115
se proceso IDD: 116
se proceso IDD: 117
se proceso IDD: 118
se proceso IDD: 119
se proceso IDD: 120
se proceso IDD: 121
se proceso IDD: 122
se proceso IDD: 123
se proceso IDD: 124
se proceso IDD: 125


In [50]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 659
se proceso ID: 660
se proceso ID: 661
se proceso ID: 662
se proceso ID: 663
se proceso ID: 664
se proceso ID: 665
se proceso ID: 666
se proceso ID: 667
se proceso ID: 25
se proceso ID: 789
se proceso ID: 670
se proceso ID: 671
se proceso ID: 672
se proceso ID: 675
se proceso ID: 805
se proceso ID: 679
se proceso ID: 680
se proceso ID: 681
se proceso ID: 682
se proceso ID: 683
se proceso ID: 684
se proceso ID: 685
se proceso ID: 686
se proceso ID: 687
se proceso ID: 688
se proceso ID: 689
se proceso ID: 690
se proceso ID: 691
se proceso ID: 692
se proceso ID: 693
se proceso ID: 694
se proceso ID: 695
se proceso ID: 696
se proceso ID: 697
se proceso ID: 698
se proceso ID: 699
se proceso ID: 700
se proceso ID: 821
se proceso ID: 851
se proceso ID: 703
se proceso ID: 852
se proceso ID: 834
se proceso ID: 836
se proceso ID: 853
se proceso ID: 839
se proceso ID: 714
se proceso ID: 842
se proceso ID: 844
se proceso ID: 845
se proceso ID: 855
se proceso ID: 591
se proceso ID

## Copiar ids de PARTE DIARIO a PLANIFICACION

In [51]:
path_planificacion = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\PLANIFICACION.shp'
path_xlsx = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [52]:
gdf_plan = gpd.read_file(path_planificacion)
df_xlsx = pd.read_excel(path_xlsx)

In [53]:
# Recorrer cada registro de df_xlsx
for index, row in df_xlsx.iterrows():
    id_value = row['ID']
    idd_value = row['IDD']
    
    # Actualizar el campo 'idd' en gdf_plan donde coincida el campo 'id'
    gdf_plan.loc[gdf_plan['id'] == id_value, 'idd'] = idd_value

In [54]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')